In [1]:
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
g = glob.glob('data/*.csv')
g

['data/gender_submission.csv', 'data/test.csv', 'data/train.csv']

In [3]:
df = pd.read_csv('data/train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Exploratory Data Analysis

### 1. Checking for missing values

#### Age

In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

High number of missing values for "Age", "Cabin", and 2 missing values for "Embarked".

In [5]:
# Age
df_age_missing = df[df.Age.isnull()]
df_age_missing.count()

PassengerId    177
Survived       177
Pclass         177
Name           177
Sex            177
Age              0
SibSp          177
Parch          177
Ticket         177
Fare           177
Cabin           19
Embarked       177
dtype: int64

In [6]:
df_age_missing.Pclass.value_counts()

3    136
1     30
2     11
Name: Pclass, dtype: int64

Mainly third class passengers who have missing ages.

In [7]:
df_age_missing.Sex.value_counts()

male      124
female     53
Name: Sex, dtype: int64

In [49]:
# Impute Age based on gender and class mean for further accuracy

means = {}
for g in ['male','female']:
    for c in [1,2,3]:
        means[f'{g}_{c}'] = df['Age'][(df.Sex == f'{g}') & (df.Pclass == c)].mean()
means

{'male_1': 41.28138613861386,
 'male_2': 30.74070707070707,
 'male_3': 26.507588932806325,
 'female_1': 34.61176470588235,
 'female_2': 28.722972972972972,
 'female_3': 21.75}

In [50]:
def impute_age(row, means):
    if pd.notnull(row['Age']):
        return row['Age']
    else:
        g = row['Sex']
        c = row['Pclass']

        return means[f'{g}_{c}']

In [51]:
df['Age'] = df.apply(impute_age, axis = 1, means = means)

In [52]:
df['Age'].isnull().sum()

0

#### Cabin

In [11]:
# not very informative if you look at the raw counts
df.Cabin.value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
F33            3
D              3
              ..
C7             1
C50            1
C148           1
B42            1
D21            1
Name: Cabin, Length: 147, dtype: int64

In [12]:
# clearer picture with just the first letter
df.Cabin.str[0].value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [212]:
pd.crosstab(columns=df.Survived, index=[df.Cabin.str[0], df.Pclass]).reset_index()

Survived,Cabin,Pclass,0,1
0,A,1,8,7
1,B,1,12,35
2,C,1,24,35
3,D,1,7,22
4,D,2,1,3
5,E,1,7,18
6,E,2,1,3
7,E,3,0,3
8,F,2,1,7
9,F,3,4,1


## Train ML model

In [229]:
from sklearn.svm import LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.compose import ColumnTransformer

In [213]:
X = df[['Sex','Age','Pclass']]
y = df['Survived']

In [214]:
X.isnull().sum()

Sex       0
Age       0
Pclass    0
dtype: int64

In [215]:
ct = ColumnTransformer([("ohe_sex", OneHotEncoder(), [0])], remainder='passthrough')
transformation_pipeline = Pipeline([('column_transformer', ct),('scaler', MinMaxScaler())])
transformation_pipeline

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex', OneHotEncoder(),
                                                  [0])])),
                ('scaler', MinMaxScaler())])

In [216]:
X = transformation_pipeline.fit_transform(X)

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=88)

In [218]:
models = [KNeighborsClassifier(), RandomForestClassifier(), LogisticRegression(), LinearSVC(), NuSVC()]

In [185]:
best_model = None
best_acc = 0

for m in models:
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    if acc > best_acc:
        best_model = m
        best_acc = acc
        print(f'Best model is now {m} with acc: {best_acc}')

Best model is now KNeighborsClassifier() with acc: 0.8156424581005587
Best model is now RandomForestClassifier() with acc: 0.8268156424581006


In [225]:
# tuning randomforest classifier
# with gridsearch
params = {
    'n_estimators':[10,50,90,100,200,300,400,500],
    'criterion':['gini', 'entropy'],
    'min_samples_split':list(range(1,10)),
    'max_depth':[None,1,4,5,10],
    'max_features':['auto','sqrt','log2']
}

In [226]:
gridsearchcv = GridSearchCV(estimator=RandomForestClassifier(), param_grid=params, scoring='precision', n_jobs=3)
gridsearchcv.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=3,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 1, 4, 5, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [10, 50, 90, 100, 200, 300, 400, 500]},
             scoring='precision')

In [227]:
gridsearchcv.best_params_

{'criterion': 'gini',
 'max_depth': 1,
 'max_features': 'auto',
 'min_samples_split': 3,
 'n_estimators': 10}

In [228]:
# test using test set
m = gridsearchcv.best_estimator_
y_pred = m.predict(X_test)
print(f"Precision: {precision_score(y_test, y_pred)}")

Precision: 0.7241379310344828


In [230]:
confusion_matrix(y_test, y_pred)

array([[101,  16],
       [ 20,  42]])

# Creation of submission

In [231]:
df_sub = pd.read_csv('data/test.csv')

In [232]:
df_sub.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [233]:
df_sub['Age'] = df_sub.apply(impute_age, axis = 1, means = means)
test_X = df_sub[['Sex','Age','Pclass']]

In [234]:
transformation_pipeline

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex', OneHotEncoder(),
                                                  [0])])),
                ('scaler', MinMaxScaler())])

In [235]:
test_X = transformation_pipeline.transform(test_X)
test_X

array([[0.        , 1.        , 0.4282483 , 1.        ],
       [1.        , 0.        , 0.58532295, 1.        ],
       [0.        , 1.        , 0.77381252, 0.5       ],
       ...,
       [0.        , 1.        , 0.47851219, 1.        ],
       [0.        , 1.        , 0.3278159 , 1.        ],
       [0.        , 1.        , 0.3278159 , 1.        ]])

In [236]:
test_y_pred = m.predict(test_X)

In [237]:
submission = pd.DataFrame({
    'PassengerId':df_sub['PassengerId'],
    'Survived':test_y_pred
})
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [238]:
submission.Survived.value_counts()

0    266
1    152
Name: Survived, dtype: int64

In [239]:
submission.to_csv('data/submission_fit_pipe.csv', index=False)